# 逐个倒排索引

In [ ]:
# 逐个索引
import os
import json
import string
import time
import re


files = sorted(["pubmed_2022d/" + ele for ele in os.listdir("pubmed_2022d") if "json" in ele])

spec_papers_d = {}
for i in ["Bactology", "Mycology", "Virology"]:
    with open("specs_list/%s.txt" % (i), "r") as f:
        specs = [i.replace("\"", "").replace("\n", "").lower() for i in f.readlines()]
    spec_paper_d = {}
    for spec in specs:
        spec_paper_d[spec] = {}
    spec_papers_d[i] = spec_paper_d

done_l = []

# 读取pubmed文件
for i, file in enumerate(files):

    # 选取没有被处理的
    if file not in done_l:
    
        print(file)

        with open(file, 'r') as f:
            papers = json.load(f)
        
        # 分割成小字典
        papers_l = []
        papers_sub = {}
        for l, (k, v) in enumerate(papers.items()):
            papers_sub[k] = v
            if l in range(999, len(papers)+1000, 1000):
                papers_l.append(papers_sub)
                papers_sub = {}
        dics = {}

        # 每个小字典
        for j, papers in enumerate(papers_l):
            
            dic = {}
            
            # 每个paper
            for k, line in papers.items():

                text = line["texts"]["Ab"] +" "+" ".join(sum(line["Meshhead"],[])) 
                paper = text.lower()
                paper = paper.translate(str.maketrans(string.punctuation, " "*32,)).replace("  ", " ")
                word_l = paper.split("")
                words = set(word_l)

                for word in words:
                    if word in dic:
                        dic[word] = dic[word] + [line]
                    else:
                        dic[word] = [line]
            
            for k, v in dic.items():
                dics.setdefault(k,[]).extend(v)

        # 每类物种检索
        for spec_type in spec_papers_d:
    
            # 每种物种
            for spec in spec_papers_d[spec_type]:
                
                terms = spec.lower().split(" ")
                term_paper = dics.setdefault(terms[0], [0])
                
                for term in terms:
                    
                    term_paper = [i for i in term_paper if i in dics.setdefault(term, [0])]

                if 0 in term_paper:
                    term_paper.remove(0)
            
            term_paper_dic = {}
            for paper in term_paper:
                term_paper_dic[term["pmid"]] = term

            spec_papers_d[spec_type][spec].update(term_paper_dic)
    
        done_l.append(file)

        if i in [300,600,900,len(files)-1]:
            with open("search_results/search_results_map.txt", 'w') as f:
                content = json.dumps(spec_papers_d)
                f.write(content)
            with open("search_results/search_done.json", 'w') as f:
                json.dump(done_l, f)

# 倒排索引建立大表

In [ ]:
# 建立词表
import os
import json
import string
import time

files = ["pubmed_2022d/" + ele for ele in os.listdir("pubmed_2022d") if "json" in ele]
files.sort()

if os.path.exists("words_dic/words_dic_done.json") is False:
    done_l = []
    print("done_l is empty")
else:
    with open("words_dic/words_dic_done.json") as f:
        done_l = json.load(f)
        print("done_l is exists")
        f.close()
invert_dic = {}
files = [i for i in files if i not in done_l]

for i, file in enumerate(files):

    print(file)
    with open(file, 'r') as f:
        papers = json.load(f)
        f.close()
            
    zerotime = time.time()
        
    papers_l = []
    papers_sub = {}
    l_list = set(range(999, len(papers)+1000, 1000))
    l_list.add(len(papers)-1)
    for l, (k, v) in enumerate(papers.items()):
        papers_sub[k] = v
        if l in l_list:
            papers_l.append(papers_sub)
            papers_sub = {}
    print(len(papers), len(papers_l))
    
    for j, papers in enumerate(papers_l):
        
        dic = {}

        for k, line in papers.items():

            text = line["texts"]["Ab"] +" "+" ".join(sum(line["Meshhead"],[])) # 添加mesh词汇
            paper = text.lower()
            paper = paper.translate(str.maketrans(string.punctuation, " "*32,)).replace("  ", " ")
            words = set(paper.split(" "))

            for word in words:
        
                dic.setdefault(word,[]).extend([k])
                
        for k, v in dic.items():
            invert_dic.setdefault(k,[]).extend(v)
        
    endtime = time.time()
    print(endtime-zerotime)
    print(len(invert_dic))
    done_l.append(file)
    
    iter_list = set(range(99, len(files), 100))
    iter_list.add(len(files)-1)
    if i in iter_list:
        
        with open("words_dic/words_dic_%s.txt" % len(done_l), 'w') as f:
            b = json.dumps(invert_dic)
            f.write(b)
        with open("words_dic/words_dic_done.json", 'w') as f:
            json.dump(done_l, f)
        invert_dic = {}

In [1]:
# 检索
import json
spec_d = {}
spec_papers_d = {}
for i in ["Bactology", "Mycology", "Virology"]:

    with open("specs_list/%s.txt" % (i), "r") as f:
        specs = [i.replace("\"", "").replace("\n", "").lower() for i in f.readlines()]
    
    spec_paper_d = {}
    for spec in specs:
        spec_paper_d[spec] = []
    spec_papers_d[i] = spec_paper_d

iter_list = set(range(100, 1114, 100))
iter_list.add(1114)
for i in iter_list:

    with open("words_dic/words_dic_%s.txt" % i, "r") as f:
        content = f.read()
        words_dic = json.loads(content)
    print("read: words_dic_%s.txt" % i)
    
    for k,v in words_dic.items():
        words_dic[k] = set(v)
        
    for spec_type in spec_papers_d:
                    
        spec_paper_d = spec_papers_d[spec_type]
        
        for spec in spec_paper_d: 

            term = spec.lower().split(" ")
            if spec_type in ["Bactology", "Mycology"]:
                if len(term) > 1:
                    term_set = words_dic.setdefault(term[0][0], {0}) | words_dic.setdefault(term[0], {0})
                else:
                    term_set = words_dic.setdefault(term[0], {0})
            else:
                term_set = words_dic.setdefault(term[0], {0})

            for j in range(1,len(term)):
                term_set = term_set & words_dic.setdefault(term[j],{0})

            spec_paper_d[spec] = spec_paper_d[spec]+ list(term_set)
        
        spec_papers_d[spec_type] = spec_paper_d

    print("search finished: words_dic_%s.txt" % i)

    del(words_dic)
    with open("search_results/search_results.txt", "w") as f:
        a = json.dumps(spec_papers_d)
        f.write(a)


read: words_dic_800.txt
search finished: words_dic_800.txt
read: words_dic_100.txt
search finished: words_dic_100.txt
read: words_dic_900.txt
search finished: words_dic_900.txt
read: words_dic_200.txt
search finished: words_dic_200.txt
read: words_dic_1000.txt
search finished: words_dic_1000.txt
read: words_dic_300.txt
search finished: words_dic_300.txt
read: words_dic_1100.txt
search finished: words_dic_1100.txt
read: words_dic_400.txt
search finished: words_dic_400.txt
read: words_dic_500.txt
search finished: words_dic_500.txt
read: words_dic_600.txt
search finished: words_dic_600.txt
read: words_dic_1114.txt
search finished: words_dic_1114.txt
read: words_dic_700.txt
search finished: words_dic_700.txt


In [2]:
# 统计
for i in spec_papers_d:
    
    spec_count = 0
    paper_count = 0
    for j in spec_papers_d[i]:
        # if len(j.split(" "))>1:
        paper_count = paper_count + len(spec_papers_d[i][j])
        # print(j, len(spec_papers_d[i][j]))
        if len(spec_papers_d[i][j]) != 0:
            spec_count = spec_count + 1
    print(len(spec_papers_d[i]), spec_count,paper_count)

1611 1602 2994859
710 703 579919
1270 1094 1842629


In [1]:
# 索引
import os
import glob
import json

with open("search_results/search_results.txt", "r") as f:
    content = f.read()
    spec_papers_d = json.loads(content)

spec_papers_dic = {}
for spec_type in spec_papers_d:
    spec_paper_d = {}
    for spec in spec_papers_d[spec_type]:
        spec_paper_d[spec] = {}
    spec_papers_dic[spec_type] = spec_paper_d

pubmeds = [i.replace(".txt", "") for i in os.listdir("pubmed_2022_merge") if "txt" in i]

for pubmed in pubmeds:
    
    with open("pubmed_2022_merge/%s.txt" % pubmed, "r") as f:
        content = f.read()
        pubmed_dic = json.loads(content)

        print(pubmed)
    
    for spec_type in spec_papers_d:        
        
        for spec in spec_papers_d[spec_type]:
        
            papers_id = set(spec_papers_d[spec_type][spec]) & pubmed_dic.keys()
            papers = {}
            for paper_id in papers_id:
                papers[pubmed_dic[paper_id]["pmid"]] = pubmed_dic[paper_id]
                
            spec_papers_dic[spec_type][spec].update(papers)
    
    del(pubmed_dic)
    
    with open("search_results/search_results_map.txt", "w") as f:
        content = json.dumps(spec_papers_dic)
        f.write(content)

pubmed_700
pubmed_400
pubmed_1100
pubmed_100
pubmed_500
pubmed_1000
pubmed_900
pubmed_600
pubmed_800
pubmed_1114
pubmed_300
pubmed_200
